In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F
import json

In [ ]:
model_path = "models/finetuned_scibert_scivocab_uncased_8cats"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [ ]:
title = "New insights into quantum computing hardware"
abstract = "This paper explores the scalability of qubit interactions in next-gen quantum processors..."
text = f"{title}. {abstract}"  # concatenate if you trained on both

In [ ]:
text = "A note on three consecutive powerful numbers. This note concerns the non-existence of three consecutive powerful numbers. We use Pell equations, elliptic curves, and second-order recurrences to show that there are no such triplets with the middle term a perfect cube and each of the other two having only a single prime factor raised to an odd power."

In [ ]:
text

In [ ]:
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length = 256)
inputs

In [ ]:
# 4. Predict
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    probabilities = F.softmax(logits, dim=-1)
    predicted_class = torch.argmax(probabilities, dim=1).item()

In [ ]:
probabilities

In [ ]:
with open("models/finetuned_scibert_scivocab_uncased_8cats/id2label.json", "r") as f:
    id2label = {int(k): v for k, v in json.load(f).items()}


id2label

In [ ]:
print(f"Predicted label: {id2label[predicted_class]}")